# Video Explanation

I creted this

[https://www.youtube.com/watch?v=7VSuYIizrdA](https://www.youtube.com/watch?v=7VSuYIizrdA)

In [ ]:
import kagglehub

path = kagglehub.dataset_download("jessicali9530/caltech256")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/caltech256


# With self large pre-trainind model

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf

# Load the entire model
base_model = load_model('1000-classes-model.h5')

In [ ]:
from keras.utils import image_dataset_from_directory

"""
split train, and test dataset, there are tensors of 32 images batch
each element of this tensor have element[0] = tensor of 32 images with (256,256,3) shape
and element[1] = tensor of 32 interger values these are classes those respective image belong
"""
(train_dataset_org, test_dataset_org) = image_dataset_from_directory(
    directory = path + '/256_ObjectCategories',
    validation_split = 0.2,
    subset= 'both', # return both train, and test
    labels = 'inferred',
    label_mode = 'categorical',
    batch_size = 32, # how many images to be considered per element of this dataset
    image_size = (256, 256),
    seed = 123
)

from tensorflow import cast, float32, one_hot

# you know these images pixel range from 0 to 255, so you need to scale it down b/w 0 and 1.
train_dataset = train_dataset_org.map(lambda x, y: (cast(x / 255.0, float32), y))
test_dataset = test_dataset_org.map(lambda x, y: (cast(x / 255.0, float32), y))

Found 30607 files belonging to 257 classes.
Using 24486 files for training.
Using 6121 files for validation.


In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import TopKCategoricalAccuracy

metrics=[
    'accuracy',  # Top-1 accuracy
    TopKCategoricalAccuracy(k=2, name='top_2_accuracy'),
    TopKCategoricalAccuracy(k=5, name='top_5_accuracy'),
]

x = base_model.layers[-3].output
x = Flatten()(x)
x = Dense(1000, activation='tanh')(x)
new_output = Dense(257, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=new_output)

for layer in model.layers[:-1]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-2), loss='categorical_crossentropy', metrics=metrics)

In [ ]:
model.fit(train_dataset, validation_data=test_dataset, epochs=2)

Epoch 1/2
766/766 ━━━━━━━━━━━━━━━━━━━━ 71s 86ms/step - accuracy: 0.2257 - loss: 7.1896 - top_2_accuracy: 0.2866 - top_5_accuracy: 0.3773 - val_accuracy: 0.3880 - val_loss: 6.7481 - val_top_2_accuracy: 0.4808 - val_top_5_accuracy: 0.6041
Epoch 2/2
766/766 ━━━━━━━━━━━━━━━━━━━━ 63s 67ms/step - accuracy: 0.4376 - loss: 6.0757 - top_2_accuracy: 0.5398 - top_5_accuracy: 0.6660 - val_accuracy: 0.4208 - val_loss: 7.6368 - val_top_2_accuracy: 0.5092 - val_top_5_accuracy: 0.6260


In [ ]:
model.fit(train_dataset, validation_data=test_dataset, epochs=6, initial_epoch=2)

Epoch 3/6
766/766 ━━━━━━━━━━━━━━━━━━━━ 51s 67ms/step - accuracy: 0.4789 - loss: 5.9468 - top_2_accuracy: 0.5923 - top_5_accuracy: 0.7293 - val_accuracy: 0.4088 - val_loss: 8.9243 - val_top_2_accuracy: 0.4999 - val_top_5_accuracy: 0.6314
Epoch 4/6
766/766 ━━━━━━━━━━━━━━━━━━━━ 51s 67ms/step - accuracy: 0.5119 - loss: 6.0122 - top_2_accuracy: 0.6260 - top_5_accuracy: 0.7587 - val_accuracy: 0.4262 - val_loss: 9.2595 - val_top_2_accuracy: 0.5246 - val_top_5_accuracy: 0.6476
Epoch 5/6
766/766 ━━━━━━━━━━━━━━━━━━━━ 53s 69ms/step - accuracy: 0.5335 - loss: 5.9687 - top_2_accuracy: 0.6531 - top_5_accuracy: 0.7903 - val_accuracy: 0.4223 - val_loss: 10.4800 - val_top_2_accuracy: 0.5198 - val_top_5_accuracy: 0.6389
Epoch 6/6
766/766 ━━━━━━━━━━━━━━━━━━━━ 81s 68ms/step - accuracy: 0.5606 - loss: 5.8163 - top_2_accuracy: 0.6764 - top_5_accuracy: 0.8094 - val_accuracy: 0.4282 - val_loss: 11.3074 - val_top_2_accuracy: 0.5238 - val_top_5_accuracy: 0.6456


In [ ]:
model.save("257-classes-model.h5")

# With Xception as Base Model

In [ ]:
from tensorflow.keras.applications.xception import Xception

base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
base_model.trainable = False

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential

model2 = Sequential()

model2.add(base_model)

model2.add(Flatten())
model2.add(Dense(257, activation='softmax'))

model2.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model2.fit(train_dataset, validation_data=test_dataset, epochs=10)

Epoch 1/10
2449/2449 ━━━━━━━━━━━━━━━━━━━━ 298s 116ms/step - accuracy: 0.6557 - loss: 2.5284 - val_accuracy: 0.7932 - val_loss: 2.0187
Epoch 2/10
2449/2449 ━━━━━━━━━━━━━━━━━━━━ 238s 97ms/step - accuracy: 0.9539 - loss: 0.2831 - val_accuracy: 0.7966 - val_loss: 2.5003
Epoch 3/10
2449/2449 ━━━━━━━━━━━━━━━━━━━━ 263s 98ms/step - accuracy: 0.9631 - loss: 0.2287 - val_accuracy: 0.7964 - val_loss: 2.8537
Epoch 4/10
 799/2449 ━━━━━━━━━━━━━━━━━━━━ 2:12 80ms/step - accuracy: 0.9794 - loss: 0.1377

KeyboardInterrupt: 